In [21]:
import pandas as pd
import numpy as np
import os

### Compilation of all appropriate CSV files

**More info to be added soon

In [50]:
#cleanup
path_to_parent = "/CSV"
current_path = os.getcwd()

# Initializing placeholder dataframe
copy_df = pd.DataFrame()

# Initializing dicts
balance_sheet_df_collection = {}
cashflow_statement_df_collection = {}
income_statement_df_collection = {}
market_stats_df_collection = {}

# Initializing dfs
balance_sheet_df = pd.DataFrame()
cashflow_statement_df = pd.DataFrame()
income_statement_df = pd.DataFrame()
market_stats_df = pd.DataFrame()

#Iterate through balance sheets quarterly
for folder in os.listdir(current_path+path_to_parent):
    for csv_file in os.listdir(os.path.join(current_path+path_to_parent, folder)):

        path_name = current_path+path_to_parent+'/'+folder+'/'+csv_file
       
        if csv_file == "balance_sheet_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            balance_sheet_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "cashflow_statement_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            cashflow_statement_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "income_statement_quarterly.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            income_statement_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        elif csv_file == "market_stats.csv":
            # Setting up a transient dataframe that gets rewritten with every iteration
            copy_df = pd.read_csv(path_name)
            # Setting up index for financial institutions
            copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
            # Setting up all columns to create a dictionary of corresponding dataframes
            column_names = copy_df.columns
            market_stats_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
        else:
            pass


balance_sheet_df = pd.concat(balance_sheet_df_collection)
cashflow_statement_df = pd.concat(cashflow_statement_df_collection)
income_statement_df = pd.concat(income_statement_df_collection)
market_stats_df = pd.concat(market_stats_df_collection)

#balance_sheet_df.head(200)
#cashflow_statement_df.head(200)
#income_statement_df.head(200)
#market_stats_df.head(200)


# Saves compiled dataframes as csv files under compiled_CSV
os.makedirs('compiled_CSV', exist_ok=True)  
balance_sheet_df.to_csv('compiled_CSV/balance_sheet.csv') 
cashflow_statement_df.to_csv('compiled_CSV/cashflow_statement.csv') 
income_statement_df.to_csv('compiled_CSV/income_statement.csv') 
market_stats_df.to_csv('compiled_CSV/market_stats.csv') 

#def create_balance_sheet_df(path_name, folder, csv_file):
#    if csv_file == "balance_sheet_quarterly.csv":
#        # Setting up a transient dataframe that gets rewritten with every iteration
#        copy_df = pd.read_csv(path_name)
#        # Setting up index for financial institutions
#        copy_df["Unnamed: 0"] = (copy_df["Unnamed: 0"] >= 0).astype(int).replace(1,folder)
#        # Setting up all columns to create a dictionary of corresponding dataframes
#        column_names = copy_df.columns
#        balance_sheet_df_collection[folder] = pd.DataFrame(copy_df, columns= column_names)
#    return balance_sheet_df_collection




In [51]:
balance_sheet_df

Unnamed: 0   Breakdown Total Assets   
EWBC  0         EWBC   3/31/2023   67,244,898  \
      1         EWBC  12/31/2022   64,112,150   
      2         EWBC   9/30/2022   62,576,061   
      3         EWBC   6/30/2022   62,394,283   
      4         EWBC   3/31/2022   62,241,456   
...              ...         ...          ...   
SIVBQ 121      SIVBQ   9/30/1992    1,025,700   
      122      SIVBQ   6/30/1992      915,900   
      123      SIVBQ   3/31/1992      899,800   
      124      SIVBQ  12/31/1991      869,500   
      125      SIVBQ   9/30/1991      869,200   

          Total Liabilities Net Minority Interest   
EWBC  0                                60,935,567  \
      1                                58,127,538   
      2                                56,915,393   
      3                                56,784,801   
      4                                56,538,000   
...                                           ...   
SIVBQ 121                                 960,500   
      122                                 841,300   
      123                                 828,900   
      124                                 803,100   
      125                                 806,300   

          Total Equity Gross Minority Interest Total Capitalization   
EWBC  0                              6,309,331            6,461,798  \
      1                              5,984,612            6,137,012   
      2                              5,660,668            6,138,198   
      3                              5,609,482            5,936,921   
      4                              5,703,456            5,930,302   
...                                        ...                  ...   
SIVBQ 121                               65,200                    -   
      122                               74,600                    -   
      123                               70,900                    -   
      124                               66,400                    -   
      125                               62,900                    -   

          Preferred Stock Equity Common Stock Equity   
EWBC  0                        -           6,309,331  \
      1                        -           5,984,612   
      2                        -           5,660,668   
      3                        -           5,609,482   
      4                        -           5,703,456   
...                          ...                 ...   
SIVBQ 121                      -              65,200   
      122                      -              74,600   
      123                      -              70,900   
      124                      -              66,400   
      125                      -              62,900   

          Capital Lease Obligations Net Tangible Assets Invested Capital   
EWBC  0                     112,676           5,843,634        4,652,467  \
      1                     111,931           5,518,915          152,400   
      2                     113,477           5,194,971          477,530   
      3                     115,387           5,143,785          327,439   
      4                     109,656           5,237,759          226,846   
...                             ...                 ...              ...   
SIVBQ 121                         -              65,200                -   
      122                         -              74,600                -   
      123                         -              70,900                -   
      124                         -              66,400                -   
      125                         -              62,900                -   

          Tangible Book Value Total Debt Net Debt Share Issued   
EWBC  0             5,843,634  4,765,143        -      141,395  \
      1             5,518,915    264,331        -      168,459   
      2             5,194,971    591,007        -      168,427   
      3             5,143,785    442,826        -      168,427   
      4        

### Clean up of compiled CSV files

 - balance_sheet_df
 - cashflow_statement_df
 - income_statement_df
 - market_stats_df

In [47]:

balance_sheet_df.rename(columns = {"Unnamed: 0":"Ticker", "Breakdown":"Date"}, inplace = True)

cols=[i for i in balance_sheet_df.columns if i not in ["Ticker","Date"]]
for col in cols:
    #balance_sheet_df[col].replace('-', np.nan, inplace = True)
    balance_sheet_df[col] = balance_sheet_df[col].astype(str).str.replace(',','')
    balance_sheet_df[col]=pd.to_numeric(balance_sheet_df[col])
    

#df[cols] = df[cols].apply(lambda x: pd.to_numeric(x.astype(str).str.replace(',',''), errors='coerce'))

balance_sheet_df
balance_sheet_df.head(130)


Ticker        Date Total Assets   
EWBC 0    EWBC   3/31/2023   67,244,898  \
     1    EWBC  12/31/2022   64,112,150   
     2    EWBC   9/30/2022   62,576,061   
     3    EWBC   6/30/2022   62,394,283   
     4    EWBC   3/31/2022   62,241,456   
...        ...         ...          ...   
MS   27     MS   6/30/2016  828,873,000   
     28     MS   3/31/2016  807,497,000   
     29     MS  12/31/2015  787,465,000   
     30     MS   9/30/2015  834,113,000   
     31     MS   6/30/2015  825,755,000   

        Total Liabilities Net Minority Interest   
EWBC 0                               60,935,567  \
     1                               58,127,538   
     2                               56,915,393   
     3                               56,784,801   
     4                               56,538,000   
...                                         ...   
MS   27                             750,498,000   
     28                             730,322,000   
     29                             711,281,000   
     30                             757,693,000   
     31                             749,688,000   

        Total Equity Gross Minority Interest Total Capitalization   
EWBC 0                             6,309,331            6,461,798  \
     1                             5,984,612            6,137,012   
     2                             5,660,668            6,138,198   
     3                             5,609,482            5,936,921   
     4                             5,703,456            5,930,302   
...                                      ...                  ...   
MS   27                           78,375,000          249,065,000   
     28                           77,175,000          246,757,000   
     29                           76,184,000          236,979,000   
     30                           76,420,000          244,068,000   
     31                           76,067,000          242,925,000   

        Preferred Stock Equity Common Stock Equity Capital Lease Obligations   
EWBC 0                     NaN           6,309,331                   112,676  \
     1                     NaN           5,984,612                   111,931   
     2                     NaN           5,660,668                   113,477   
     3                     NaN           5,609,482                   115,387   
     4                     NaN           5,703,456                   109,656   
...                        ...                 ...                       ...   
MS   27              7,520,000          69,596,000                18,738,000   
     28              7,520,000          68,490,000                       NaN   
     29              7,520,000          67,662,000                       NaN   
     30              7,520,000          67,767,000                       NaN   
     31              7,520,000          67,518,000                       NaN   

        Net Tangible Assets Invested Capital Tangible Book Value   Total Debt   
EWBC 0            5,843,634        4,652,467           5,843,634    4,765,143  \
     1            5,518,915          152,400           5,518,915      264,331   
     2            5,194,971          477,530           5,194,971      591,007   
     3            5,143,785          327,439           5,143,785      442,826   
     4            5,237,759          226,846           5,237,759      336,502   
...                     ...              ...                 ...          ...   
MS   27          67,702,000      243,869,000          60,182,000  174,273,000   
     28          66,515,000      241,719,000          58,995,000  173,229,000   
     29          65,614,000      233,067,000          58,094,000  165,405,000   
     30          65,631,000      240,263,000          58,111,000  172,496,000   
     31          65,293,000      240,417,000          57,773,000  172,899,000   

            Net Debt Share Issued Ordinary Shares Number   
EWBC 0           NaN      141,395                141,39

In [48]:
#print(balance_sheet_df.apply(type))
balance_sheet_df.dtypes

Ticker                                     object
Date                                       object
Total Assets                               object
Total Liabilities Net Minority Interest    object
Total Equity Gross Minority Interest       object
Total Capitalization                       object
Preferred Stock Equity                     object
Common Stock Equity                        object
Capital Lease Obligations                  object
Net Tangible Assets                        object
Invested Capital                           object
Tangible Book Value                        object
Total Debt                                 object
Net Debt                                   object
Share Issued                               object
Ordinary Shares Number                     object
Preferred Shares Number                    object
Treasury Shares Number                     object
dtype: object

In [28]:
market_stats_df["Unnamed: 0"].unique()

array(['EWBC', 'MS', 'MCBC', 'Bk', 'GS', 'DFS', 'WAL', 'WFC', 'BAC', 'CS',
       'PACW', 'KEY', 'COF', 'TD', 'CFG', 'NYCB', 'CMA', 'HSBC', 'SBNY',
       'ZION', 'DB', 'MTB', 'FRCB', 'FHN', 'C', 'FCNCA', 'OVLY', 'JPM',
       'PNC', 'SIVBQ'], dtype=object)